# **Space X  Falcon 9 First Stage Landing Prediction**

In this module of the project, it is performed some Exploratory Data Analysis (EDA) using SQL

### Objectives:
Perform Exploratory  Data Analysis and determine Training Labels 
- Exploratory Data Analysis
- Preparing Data  Feature Engineering 

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [1]:
# !pip install sqlalchemy
# !pip install ipython-sql

In [3]:
import sqlite3

database = 'datasets/spaceX.db' # Has to be in the same directory of the working directory

connection = sqlite3.connect(database)
cur = connection.cursor()

In [4]:
%load_ext sql
%sql sqlite:///datasets/spaceX.db

In [5]:
import pandas as pd
# df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
# df.to_csv('datasets/spaceX_for_sql.csv')
df = pd.read_csv("datasets/spaceX_for_sql.csv", index_col=0)
df.to_sql("SPACEXTBL", connection, if_exists='replace', index=False, method="multi")

101

**Note: In this project, blank rows are removed from the table.**

In [8]:
%sql create table SPACEXTABLE as SELECT * from SPACEXTBL where Date is not null

 * sqlite:///datasets/spaceX.db
Done.


[]

#### Display the names of the unique launch sites  in the space mission

In [9]:
%sql SELECT * FROM PRAGMA_TABLE_INFO('SPACEXTABLE')

 * sqlite:///datasets/spaceX.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Date,TEXT,0,None,0
1,Time (UTC),TEXT,0,None,0
2,Booster_Version,TEXT,0,None,0
3,Launch_Site,TEXT,0,None,0
4,Payload,TEXT,0,None,0
5,PAYLOAD_MASS__KG_,INT,0,None,0
6,Orbit,TEXT,0,None,0
7,Customer,TEXT,0,None,0
8,Mission_Outcome,TEXT,0,None,0
9,Landing_Outcome,TEXT,0,None,0


In [9]:
%sql SELECT DISTINCT(Launch_Site) FROM SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


####  Display 5 records where launch sites begin with the string 'CCA' 

In [10]:
%%sql SELECT * FROM SPACEXTABLE 
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


#### Display the total payload mass carried by boosters launched by NASA (CRS)

In [11]:
%sql SELECT SUM(PAYLOAD_MASS__KG_) AS 'AVERAGE PAYLOAD MASS (KG)' FROM SPACEXTABLE WHERE Customer == 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


AVERAGE PAYLOAD MASS (KG)
45596


#### Display average payload mass carried by booster version F9 v1.1

In [12]:
%sql SELECT AVG(PAYLOAD_MASS__KG_) AS 'AVERAGE PAYLOAD MASS (KG)' FROM SPACEXTABLE WHERE Booster_Version LIKE '%%F9 V1.1%'

 * sqlite:///my_data1.db
Done.


AVERAGE PAYLOAD MASS (KG)
2534.6666666666665


#### List the date when the first succesful landing outcome in ground pad was acheived.

In [13]:
%sql SELECT * FROM SPACEXTABLE WHERE Landing_Outcome LIKE 'Success (ground pad)%' ORDER BY Date ASC LIMIT 1

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2015-12-22,1:29:00,F9 FT B1019,CCAFS LC-40,OG2 Mission 2 11 Orbcomm-OG2 satellites,2034,LEO,Orbcomm,Success,Success (ground pad)


In [14]:
%sql SELECT Date AS 'FIRST SUCCESSFUL LANDING DATE' FROM SPACEXTABLE WHERE Landing_Outcome LIKE 'Success (ground pad)%' ORDER BY Date ASC LIMIT 1

 * sqlite:///my_data1.db
Done.


FIRST SUCCESSFUL LANDING DATE
2015-12-22


#### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [21]:
%%sql SELECT DISTINCT(Booster_Version), PAYLOAD_MASS__KG_ AS 'PAYLOAD MASS (KG)' 
FROM SPACEXTABLE SPACEXTABLE WHERE Landing_Outcome LIKE 'Success (drone ship)%' 
AND PAYLOAD_MASS__KG_ > 4000 AND PAYLOAD_MASS__KG_ < 6000

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD MASS (KG)
F9 FT B1022,4696
F9 FT B1026,4600
F9 FT B1021.2,5300
F9 FT B1031.2,5200


#### List the total number of successful and failure mission outcomes

In [16]:
%sql SELECT DISTINCT(Mission_Outcome), COUNT(Mission_Outcome) FROM SPACEXTABLE GROUP BY Mission_Outcome

 * sqlite:///my_data1.db
Done.


Mission_Outcome,COUNT(Mission_Outcome)
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


#### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [17]:
%%sql SELECT Booster_Version, PAYLOAD_MASS__KG_ AS 'MAX PAYLOAD MASS (KG)' 
FROM SPACEXTABLE WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE)

 * sqlite:///my_data1.db
Done.


Booster_Version,MAX PAYLOAD MASS (KG)
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


#### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [18]:
%%sql SELECT strftime('%m', Date) AS Month, strftime('%Y', Date) AS Year, Landing_Outcome, Booster_Version, Launch_Site FROM SPACEXTABLE  
WHERE Landing_Outcome LIKE 'Failure%' AND strftime('%Y', Date) == '2015'

 * sqlite:///my_data1.db
Done.


Month,Year,Landing_Outcome,Booster_Version,Launch_Site
01,2015,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,2015,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


#### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [24]:
%%sql SELECT Date, Landing_Outcome AS 'COUNT LANDING OUTCOMES BETWEEN 2010/06/04 AND 2017/03/20' FROM SPACEXTABLE 
WHERE Landing_Outcome == 'Failure (drone ship)' OR Landing_Outcome == 'Success (ground pad)'
ORDER BY 'Landing_Outcome' DESC

 * sqlite:///my_data1.db
Done.


Date,COUNT LANDING OUTCOMES BETWEEN 2010/06/04 AND 2017/03/20
2015-01-10,Failure (drone ship)
2015-04-14,Failure (drone ship)
2015-12-22,Success (ground pad)
2016-01-17,Failure (drone ship)
2016-03-04,Failure (drone ship)
2016-06-15,Failure (drone ship)
2016-07-18,Success (ground pad)
2017-02-19,Success (ground pad)
2017-05-01,Success (ground pad)
2017-06-03,Success (ground pad)


© IBM Corporation 2021. All rights reserved.